In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn
import math

pn.extension()

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())

marketing_data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/MarketingPerProduct.csv', index_col=0)
price_per_unit = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/PricePerUnit.csv', index_col=0)
profit_per_unit = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/ProfitPerUnit.csv', index_col=0)

summary_data = pd.DataFrame(index=data.columns)
summary_data['Price'] = price_per_unit.values
summary_data['Profit'] = profit_per_unit.values
summary_data['Sales'] = data.sum().values
summary_data['Marketing'] = marketing_data.values
summary_data['Advertising'] = summary_data['Marketing'] / summary_data['Sales']
summary_data['Cost'] = summary_data['Price'] - summary_data['Profit']
print(summary_data.head())
print(summary_data.describe())

variables = list(summary_data.columns)

In [ ]:
def get_line_plot(volume, window_size, date_range):
    if volume == 'high':
        selected = ['A', 'F', 'L']
    elif volume == 'medium':
        selected = ['G', 'H', 'J', 'S', 'W']
    elif volume == 'low':
        selected = ['D', 'E', 'M', 'O', 'P', 'T', 'X']
    else:
        selected = ['B', 'C', 'I', 'K', 'N', 'Q', 'R', 'U', 'V', 'Y']
    df = data[selected] # restrict to selected products
    df = df.loc[pd.to_datetime(date_range[0]) : pd.to_datetime(date_range[1])] # restrict to date range
    df = df.rolling(window=window_size).mean() # calculate rolling average
    return df.hvplot.line(frame_height=200, frame_width=600, grid=True)


time_title = '### Time-Series'

volumes = ['high', 'medium', 'low', 'very low']

volume = pn.widgets.Select(name='Volume', options=volumes, width=200)
day_avg = pn.widgets.IntSlider(name='Day average', value=7, start=1, end=21, width=200)
time_date_range = pn.widgets.DateRangeSlider(name='Dates', width=200,
                                        value=(data.index[0], data.index[-1]),
                                        start=data.index[0], end=data.index[-1])

time_row = pn.Row(
    pn.Column(time_title, volume, day_avg, time_date_range),
    get_line_plot(volume.options[0], day_avg.value, time_date_range.value)
)

def update_time_row(event):
    time_row[1].object = get_line_plot(volume.value, day_avg.value, time_date_range.value)

volume.param.watch(update_time_row, 'value')
day_avg.param.watch(update_time_row, 'value')
time_date_range.param.watch(update_time_row, 'value')

time_row.servable()

In [ ]:
def get_correlation_plots(product_x, product_y, date_range):
    df = data.loc[pd.to_datetime(date_range[0]) : pd.to_datetime(date_range[1])] # restrict to date range
    return df.hvplot.scatter(frame_height=250, frame_width=250,
                             x=product_x, y=product_y,
                             title=product_x + ' vs ' + product_y,
                             size=10, padding=0.1) +\
           df.corr().hvplot.heatmap(frame_height=250, frame_width=250,
                                    title='Product correlations',
                                    rot=90, cmap='coolwarm'
                                   ).opts(invert_yaxis=True, clim=(-1, 1))


correlation_title = '### Correlations'

product_x = pn.widgets.Select(name='x-axis', options=list(data.columns), width=200)
product_y = pn.widgets.Select(name='y-axis', options=list(data.columns), width=200)
corr_date_range = pn.widgets.DateRangeSlider(name='Dates',
                                        value=(data.index[0], data.index[-1]),
                                        start=data.index[0], end=data.index[-1],
                                        width=200)

correlation_row = pn.Row(
    pn.Column(correlation_title, product_x, product_y, corr_date_range),
    get_correlation_plots(product_x.value, product_y.value, corr_date_range.value)
)

def update_correlation_row(event):
    correlation_row[1].object = get_correlation_plots(product_x.value, product_y.value, corr_date_range.value)

product_x.param.watch(update_correlation_row, 'value')
product_y.param.watch(update_correlation_row, 'value')
corr_date_range.param.watch(update_correlation_row, 'value')

correlation_row.servable()

In [ ]:
def get_summary_plots(variable_x, variable_y, variable_z, bubble_scaling):
    summary_data['BubbleSize'] = summary_data[variable_z] / summary_data[variable_z].max() * math.exp(bubble_scaling)
    return summary_data.hvplot.scatter(frame_height=250, frame_width=250,
                                       title=variable_x + ' vs ' + variable_y + ' (vs ' + variable_z + ')',
                                       alpha=0.5, hover_cols='all', padding=0.1,
                                       x=variable_x, y=variable_y, size='BubbleSize') 


variables_title = '### Variables'

variable_x = pn.widgets.Select(name='X-axis', options=variables, value=variables[0], width=200)
variable_y = pn.widgets.Select(name='Y-axis', options=variables, value=variables[1], width=200)
variable_z = pn.widgets.Select(name='Bubble', options=variables, value=variables[2], width=200)
bubble_scaling = pn.widgets.IntSlider(name='Bubble scaling', value=5, start=0, end=10, width=200)

variables_row = pn.Row(
    pn.Column(variables_title, variable_x, variable_y, variable_z, bubble_scaling),
    get_summary_plots(variable_x.value, variable_y.value, variable_z.value, bubble_scaling.value)
)

def update_variables_row(event):
    variables_row[1].object = get_summary_plots(variable_x.value, variable_y.value, variable_z.value, bubble_scaling.value)

variable_x.param.watch(update_variables_row, 'value')
variable_y.param.watch(update_variables_row, 'value')
variable_z.param.watch(update_variables_row, 'value')
bubble_scaling.param.watch(update_variables_row, 'value')

variables_row.servable()